In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from tqdm import tqdm
import requests
from urllib.parse import urlparse
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import datetime
from sklearn.preprocessing import quantile_transform
pd.options.mode.chained_assignment = None  # default='warn'
import gc

# Read Data

In [3]:
main_data_df = pd.read_csv('main_data_201229-141401.csv')
main_data_df.shape

(3225086, 72)

In [4]:
main_data_df.head()

,전용면적(㎡),가격(만원),층,건축년도,년,월,공시지가(만원),토지면적,지목명_대,지목명_도로,지목명_임야,지목명_공원,지목명_학교용지,지목명_종교용지,지목명_주차장,지목명_철도용지,지목명_주유소용지,지목명_잡종지,지목명_수도용지,지목명_전,지목명_공장용지,지목명_창고용지,지목명_답,용도지역명1_제1종일반주거지역,용도지역명1_자연녹지지역,용도지역명1_개발제한구역,용도지역명1_제2종일반주거지역,용도지역명1_제3종일반주거지역,용도지역명1_일반상업지역,용도지역명1_준주거지역,용도지역명1_제1종전용주거지역,용도지역명1_중심상업지역,용도지역명1_근린상업지역,용도지역명1_준공업지역,용도지역명1_유통상업지역,용도지역명1_제2종전용주거지역,지형높이_평지,지형높이_완경사,지형높이_지정되지않음,지형높이_급경사,지형높이_고지,지형높이_저지,지형형상_사다리형,지형형상_부정형,지형형상_지정되지않음,지형형상_삼각형,지형형상_정방형,지형형상_가로장방,지형형상_역삼각형,지형형상_자루형,지형형상_세로장방,도로접면_소로한면,도로접면_지정되지않음,도로접면_맹지,도로접면_중로각지,도로접면_중로한면,도로접면_소로각지,도로접면_세로한면(가),도로접면_세로한면(불),도로접면_세로각지(불),도로접면_세로각지(가),도로접면_광대로한면,도로접면_광대세각,도로접면_광대소각,x좌표,y좌표,전월세매매구분_매매,전월세매매구분_전세,전월세매매구분_월세,건물종류_아파트,건물종류_오피스텔,건물종류_연립다세대
0,77.75,57000,2,1988,2013,9,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,960471.903756,1.942550e+06,1,0,0,1,0,0
1,77.75,57000,2,1988,2013,12,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,960471.903756,1.942550e+06,1,0,0,1,0,0
2,77.75,35000,4,1988,2013,1,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,960471.903756,1.942550e+06,0,1,0,1,0,0
3,77.75,32000,9,1988,2013,3,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,960471.903756,1.942550e+06,0,1,0,1,0,0
4,77.75,37000,1,1988,2013,9,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,960471.903756,1.942550e+06,0,1,0,1,0,0


In [5]:
def read_coor_data1():
    basedir = './좌표데이터/'
    filenames = os.listdir(basedir)
    
    dfs_list = []
    
    for filename in tqdm(filenames, position=0):
        if filename.endswith('.txt'):
            try:
                df = pd.read_csv(basedir + filename, sep='|', header=None).drop_duplicates()
            except:
                df = pd.read_csv(basedir + filename, sep='|', header=None, encoding='ansi').drop_duplicates()
                            
            df['도로명주소'] = df[3] + ' ' + df[4] + ' ' + df[7] + ' ' + df[9].astype('str') + '-' + df[10].astype('str')
            
            df['x좌표'] = df[16]
            df['y좌표'] = df[17]
            
            df = df[['도로명주소', 'x좌표', 'y좌표']]
            
            dfs_list.append(df)
            
    concat_df = pd.concat(dfs_list).drop_duplicates(subset=['도로명주소'], keep='last').reset_index(drop=True)
        
    return concat_df

def read_coor_data2():
    basedir = './좌표데이터2/'
    filenames = os.listdir(basedir)
    
    dfs_list = []
    
    for filename in tqdm(filenames, position=0):
        if filename.endswith('.txt'):
            try:
                df = pd.read_csv(basedir + filename, sep='|', header=None, usecols=[1, 2, 3, 4, 5, 7, 8, 23, 24]).drop_duplicates()
            except:
                df = pd.read_csv(basedir + filename, sep='|', header=None, encoding='ansi', usecols=[1, 2, 3, 4, 5, 7, 8, 23, 24])\
                .drop_duplicates()
                
            df['도로명주소'] = df[1] + ' ' + df[2] + ' ' + df[5] + ' ' + df[7].astype('str') + '-' + df[8].astype('str')
            
            df['x좌표'] = df[23]
            df['y좌표'] = df[24]
            
            df = df[['도로명주소', 'x좌표', 'y좌표']]
            
            dfs_list.append(df)
            
    concat_df = pd.concat(dfs_list).drop_duplicates(subset=['도로명주소'], keep='last').reset_index(drop=True)
        
    return concat_df

def read_coor_data3():
    basedir = './좌표데이터3/'
    filenames = os.listdir(basedir)
    
    dfs_list = []
    
    for filename in tqdm(filenames, position=0):
        if filename.endswith('.csv'):
            df = pd.read_csv(basedir + filename, index_col=0).drop_duplicates()            
            dfs_list.append(df)
            
    concat_df = pd.concat(dfs_list).drop_duplicates(subset=['전체주소'], keep='last').reset_index(drop=True)
    concat_df.columns = ['지번주소', 'x좌표', 'y좌표']
    
    return concat_df

def add_0(x):
    if pd.isnull(x) == False:
        splitted = x.split('-')
        if len(splitted) == 1:
            return x+'-0'
        else:
            return x
    else:
        return x
    
def read_coor_data():
    roadname_coor_df1 = read_coor_data1()
    roadname_coor_df2 = read_coor_data2()
    
    roadname_coor_df = pd.concat([roadname_coor_df1, roadname_coor_df2]).drop_duplicates(subset=['도로명주소'], keep='last')\
    .reset_index(drop=True)
    
    landnum_coor_df = read_coor_data3()
        
    return roadname_coor_df, landnum_coor_df

In [6]:
roadname_coor_df, landnum_coor_df = read_coor_data()

100%|██████████████████████████████████████████████████████████████████████████████████| 31/31 [00:00<00:00, 40.14it/s]


# Feature Engineering

In [11]:
def remove_useless_cols(df):
    gc.collect()
    df = df.copy()
    description = df.describe()
    
    cols_to_drop = []
    for col in tqdm(description.columns, position=0):
        if description[col]['std'] == 0:
            cols_to_drop.append(col)
    
    df.drop(columns=cols_to_drop, inplace=True)
    
    print('{}개의 무의미한 콜럼들을 찾아서 제거했습니다.'.format(len(cols_to_drop)))
    
    return df

In [12]:
main_data_df = remove_useless_cols(main_data_df)
main_data_df.shape

100%|███████████████████████████████████████████████████████████████████████████████| 73/73 [00:00<00:00, 24383.55it/s]


0개의 무의미한 콜럼들을 찾아서 제거했습니다.


(3225086, 73)

In [8]:
def building_age(df):
    gc.collect()
    df = df.copy()
    
    df['건물나이'] = df['년'] - df['건축년도']
    
    return df

In [9]:
main_data_df = building_age(main_data_df)
main_data_df.shape

(3225086, 73)

In [10]:
main_data_df.head()

,전용면적(㎡),가격(만원),층,건축년도,년,월,공시지가(만원),토지면적,지목명_대,지목명_도로,지목명_임야,지목명_공원,지목명_학교용지,지목명_종교용지,지목명_주차장,지목명_철도용지,지목명_주유소용지,지목명_잡종지,지목명_수도용지,지목명_전,지목명_공장용지,지목명_창고용지,지목명_답,용도지역명1_제1종일반주거지역,용도지역명1_자연녹지지역,용도지역명1_개발제한구역,용도지역명1_제2종일반주거지역,용도지역명1_제3종일반주거지역,용도지역명1_일반상업지역,용도지역명1_준주거지역,용도지역명1_제1종전용주거지역,용도지역명1_중심상업지역,용도지역명1_근린상업지역,용도지역명1_준공업지역,용도지역명1_유통상업지역,용도지역명1_제2종전용주거지역,지형높이_평지,지형높이_완경사,지형높이_지정되지않음,지형높이_급경사,지형높이_고지,지형높이_저지,지형형상_사다리형,지형형상_부정형,지형형상_지정되지않음,지형형상_삼각형,지형형상_정방형,지형형상_가로장방,지형형상_역삼각형,지형형상_자루형,지형형상_세로장방,도로접면_소로한면,도로접면_지정되지않음,도로접면_맹지,도로접면_중로각지,도로접면_중로한면,도로접면_소로각지,도로접면_세로한면(가),도로접면_세로한면(불),도로접면_세로각지(불),도로접면_세로각지(가),도로접면_광대로한면,도로접면_광대세각,도로접면_광대소각,x좌표,y좌표,전월세매매구분_매매,전월세매매구분_전세,전월세매매구분_월세,건물종류_아파트,건물종류_오피스텔,건물종류_연립다세대,건물나이
0,77.75,57000,2,1988,2013,9,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,960471.903756,1.942550e+06,1,0,0,1,0,0,25
1,77.75,57000,2,1988,2013,12,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,960471.903756,1.942550e+06,1,0,0,1,0,0,25
2,77.75,35000,4,1988,2013,1,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,960471.903756,1.942550e+06,0,1,0,1,0,0,25
3,77.75,32000,9,1988,2013,3,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,960471.903756,1.942550e+06,0,1,0,1,0,0,25
4,77.75,37000,1,1988,2013,9,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,960471.903756,1.942550e+06,0,1,0,1,0,0,25


In [13]:
def month_circular_sine_func(x):
    return math.sin(2*math.pi*((x-1)/12))

def month_circular_cosine_func(x):
    return math.cos(2*math.pi*((x-1)/12))

In [14]:
def time_feature_engineering(df):
    gc.collect()
    df = df.copy()
    
    df['time_linear'] = df['년'] - 2012 + ((df['월'] - 1) / 12)
    
    df['month_circular_sine'] = df['월'].apply(month_circular_sine_func)
    df['month_circular_cosine'] = df['월'].apply(month_circular_cosine_func)
    
    return df

In [15]:
main_data_df = time_feature_engineering(main_data_df)
main_data_df.shape

(3225086, 76)

In [23]:
main_data_df.head()

,전용면적(㎡),가격(만원),층,건축년도,년,월,공시지가(만원),토지면적,지목명_대,지목명_도로,지목명_임야,지목명_공원,지목명_학교용지,지목명_종교용지,지목명_주차장,지목명_철도용지,지목명_주유소용지,지목명_잡종지,지목명_수도용지,지목명_전,지목명_공장용지,지목명_창고용지,지목명_답,용도지역명1_제1종일반주거지역,용도지역명1_자연녹지지역,용도지역명1_개발제한구역,용도지역명1_제2종일반주거지역,용도지역명1_제3종일반주거지역,용도지역명1_일반상업지역,용도지역명1_준주거지역,용도지역명1_제1종전용주거지역,용도지역명1_중심상업지역,용도지역명1_근린상업지역,용도지역명1_준공업지역,용도지역명1_유통상업지역,용도지역명1_제2종전용주거지역,지형높이_평지,지형높이_완경사,지형높이_지정되지않음,지형높이_급경사,지형높이_고지,지형높이_저지,지형형상_사다리형,지형형상_부정형,지형형상_지정되지않음,지형형상_삼각형,지형형상_정방형,지형형상_가로장방,지형형상_역삼각형,지형형상_자루형,지형형상_세로장방,도로접면_소로한면,도로접면_지정되지않음,도로접면_맹지,도로접면_중로각지,도로접면_중로한면,도로접면_소로각지,도로접면_세로한면(가),도로접면_세로한면(불),도로접면_세로각지(불),도로접면_세로각지(가),도로접면_광대로한면,도로접면_광대세각,도로접면_광대소각,x좌표,y좌표,전월세매매구분_매매,전월세매매구분_전세,전월세매매구분_월세,건물종류_아파트,건물종류_오피스텔,건물종류_연립다세대,건물나이,time_linear,month_circular_sine,month_circular_cosine
0,77.75,57000,2,1988,2013,9,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,960471.903756,1.942550e+06,1,0,0,1,0,0,25,1.666667,-0.866025,-0.500000
1,77.75,57000,2,1988,2013,12,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,960471.903756,1.942550e+06,1,0,0,1,0,0,25,1.916667,-0.500000,0.866025
2,77.75,35000,4,1988,2013,1,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,960471.903756,1.942550e+06,0,1,0,1,0,0,25,1.000000,0.000000,1.000000
3,77.75,32000,9,1988,2013,3,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,960471.903756,1.942550e+06,0,1,0,1,0,0,25,1.166667,0.866025,0.500000
4,77.75,37000,1,1988,2013,9,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,960471.903756,1.942550e+06,0,1,0,1,0,0,25,1.666667,-0.866025,-0.500000


In [19]:
def area_ratios(df):
    gc.collect()
    df = df.copy()
    
    df['전용/토지면적비율'] = df['전용면적(㎡)'] / df['토지면적']
    
    return df

In [24]:
cols_to_drop = ['건축년도', '년', '월']
main_data_df.drop(columns=cols_to_drop, inplace=True)

In [25]:
print(main_data_df.shape)
main_data_df.head()

(3225086, 73)


,전용면적(㎡),가격(만원),층,공시지가(만원),토지면적,지목명_대,지목명_도로,지목명_임야,지목명_공원,지목명_학교용지,지목명_종교용지,지목명_주차장,지목명_철도용지,지목명_주유소용지,지목명_잡종지,지목명_수도용지,지목명_전,지목명_공장용지,지목명_창고용지,지목명_답,용도지역명1_제1종일반주거지역,용도지역명1_자연녹지지역,용도지역명1_개발제한구역,용도지역명1_제2종일반주거지역,용도지역명1_제3종일반주거지역,용도지역명1_일반상업지역,용도지역명1_준주거지역,용도지역명1_제1종전용주거지역,용도지역명1_중심상업지역,용도지역명1_근린상업지역,용도지역명1_준공업지역,용도지역명1_유통상업지역,용도지역명1_제2종전용주거지역,지형높이_평지,지형높이_완경사,지형높이_지정되지않음,지형높이_급경사,지형높이_고지,지형높이_저지,지형형상_사다리형,지형형상_부정형,지형형상_지정되지않음,지형형상_삼각형,지형형상_정방형,지형형상_가로장방,지형형상_역삼각형,지형형상_자루형,지형형상_세로장방,도로접면_소로한면,도로접면_지정되지않음,도로접면_맹지,도로접면_중로각지,도로접면_중로한면,도로접면_소로각지,도로접면_세로한면(가),도로접면_세로한면(불),도로접면_세로각지(불),도로접면_세로각지(가),도로접면_광대로한면,도로접면_광대세각,도로접면_광대소각,x좌표,y좌표,전월세매매구분_매매,전월세매매구분_전세,전월세매매구분_월세,건물종류_아파트,건물종류_오피스텔,건물종류_연립다세대,건물나이,time_linear,month_circular_sine,month_circular_cosine
0,77.75,57000,2,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,960471.903756,1.942550e+06,1,0,0,1,0,0,25,1.666667,-0.866025,-0.500000
1,77.75,57000,2,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,960471.903756,1.942550e+06,1,0,0,1,0,0,25,1.916667,-0.500000,0.866025
2,77.75,35000,4,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,960471.903756,1.942550e+06,0,1,0,1,0,0,25,1.000000,0.000000,1.000000
3,77.75,32000,9,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,960471.903756,1.942550e+06,0,1,0,1,0,0,25,1.166667,0.866025,0.500000
4,77.75,37000,1,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,960471.903756,1.942550e+06,0,1,0,1,0,0,25,1.666667,-0.866025,-0.500000


In [2]:
main_data_df = pd.read_csv('training_data.csv')

In [3]:
main_data_df.head()

,전용면적(㎡),가격(만원),층,공시지가(만원),토지면적,지목명_대,지목명_도로,지목명_임야,지목명_공원,지목명_학교용지,지목명_종교용지,지목명_주차장,지목명_철도용지,지목명_주유소용지,지목명_잡종지,지목명_수도용지,지목명_전,지목명_공장용지,지목명_창고용지,지목명_답,용도지역명1_제1종일반주거지역,용도지역명1_자연녹지지역,용도지역명1_개발제한구역,용도지역명1_제2종일반주거지역,용도지역명1_제3종일반주거지역,용도지역명1_일반상업지역,용도지역명1_준주거지역,용도지역명1_제1종전용주거지역,용도지역명1_중심상업지역,용도지역명1_근린상업지역,용도지역명1_준공업지역,용도지역명1_유통상업지역,용도지역명1_제2종전용주거지역,지형높이_평지,지형높이_완경사,지형높이_지정되지않음,지형높이_급경사,지형높이_고지,지형높이_저지,지형형상_사다리형,지형형상_부정형,지형형상_지정되지않음,지형형상_삼각형,지형형상_정방형,지형형상_가로장방,지형형상_역삼각형,지형형상_자루형,지형형상_세로장방,도로접면_소로한면,도로접면_지정되지않음,도로접면_맹지,도로접면_중로각지,도로접면_중로한면,도로접면_소로각지,도로접면_세로한면(가),도로접면_세로한면(불),도로접면_세로각지(불),도로접면_세로각지(가),도로접면_광대로한면,도로접면_광대세각,도로접면_광대소각,x좌표,y좌표,전월세매매구분_매매,전월세매매구분_전세,전월세매매구분_월세,건물종류_아파트,건물종류_오피스텔,건물종류_연립다세대,건물나이,time_linear,month_circular_sine,month_circular_cosine
0,77.75,57000,2,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,960471.903756,1.942550e+06,1,0,0,1,0,0,25,1.666667,-0.866025,-0.500000
1,77.75,57000,2,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,960471.903756,1.942550e+06,1,0,0,1,0,0,25,1.916667,-0.500000,0.866025
2,77.75,35000,4,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,960471.903756,1.942550e+06,0,1,0,1,0,0,25,1.000000,0.000000,1.000000
3,77.75,32000,9,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,960471.903756,1.942550e+06,0,1,0,1,0,0,25,1.166667,0.866025,0.500000
4,77.75,37000,1,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,960471.903756,1.942550e+06,0,1,0,1,0,0,25,1.666667,-0.866025,-0.500000


In [11]:
roadname_coor_df.rename(columns={'도로명주소':'주소'}, inplace=True)
landnum_coor_df.rename(columns={'지번주소':'주소'}, inplace=True)

In [15]:
coor_df = pd.concat([roadname_coor_df, landnum_coor_df]).reset_index(drop=True)

In [16]:
coor_df.head()

,주소,x좌표,y좌표
0,서울특별시 관악구 난곡로24가길 18-0,948851.369529,1.941318e+06
1,서울특별시 종로구 성균관로15길 33-0,955591.635372,1.954533e+06
2,서울특별시 성북구 인촌로7길 70-0,957563.072605,1.954475e+06
3,서울특별시 서초구 강남대로91길 5-0,957617.535913,1.945960e+06
4,서울특별시 서초구 바우뫼로11길 54-0,958121.727726,1.941641e+06


In [44]:
def coor_normalization(df, coor_df):
    gc.collect()
    df = df.copy()
    coor_df = coor_df.copy()
    
    x_min = df['x좌표'].min()
    x_max = df['x좌표'].max()
    x_range = x_max - x_min
    x_middle = x_range /2
    
    y_min = df['y좌표'].min()
    y_max = df['y좌표'].max()
    y_range = y_max - y_min
    y_middle = y_range /2
    
    x_range_ratio = x_range / y_range
    
    print(x_range, y_range)
    
    df['x_norm'] = ((df['x좌표'] - x_middle) / 100)
    df['y_norm'] = ((df['y좌표'] - y_middle) / 100)
    
    return df

In [35]:
main_data_df = coor_normalization(main_data_df, coor_df)

33771.050118000014 28464.324538999936


In [36]:
main_data_df.shape

(3225086, 75)

In [37]:
main_data_df.head()

,전용면적(㎡),가격(만원),층,공시지가(만원),토지면적,지목명_대,지목명_도로,지목명_임야,지목명_공원,지목명_학교용지,지목명_종교용지,지목명_주차장,지목명_철도용지,지목명_주유소용지,지목명_잡종지,지목명_수도용지,지목명_전,지목명_공장용지,지목명_창고용지,지목명_답,용도지역명1_제1종일반주거지역,용도지역명1_자연녹지지역,용도지역명1_개발제한구역,용도지역명1_제2종일반주거지역,용도지역명1_제3종일반주거지역,용도지역명1_일반상업지역,용도지역명1_준주거지역,용도지역명1_제1종전용주거지역,용도지역명1_중심상업지역,용도지역명1_근린상업지역,용도지역명1_준공업지역,용도지역명1_유통상업지역,용도지역명1_제2종전용주거지역,지형높이_평지,지형높이_완경사,지형높이_지정되지않음,지형높이_급경사,지형높이_고지,지형높이_저지,지형형상_사다리형,지형형상_부정형,지형형상_지정되지않음,지형형상_삼각형,지형형상_정방형,지형형상_가로장방,지형형상_역삼각형,지형형상_자루형,지형형상_세로장방,도로접면_소로한면,도로접면_지정되지않음,도로접면_맹지,도로접면_중로각지,도로접면_중로한면,도로접면_소로각지,도로접면_세로한면(가),도로접면_세로한면(불),도로접면_세로각지(불),도로접면_세로각지(가),도로접면_광대로한면,도로접면_광대세각,도로접면_광대소각,x좌표,y좌표,전월세매매구분_매매,전월세매매구분_전세,전월세매매구분_월세,건물종류_아파트,건물종류_오피스텔,건물종류_연립다세대,건물나이,time_linear,month_circular_sine,month_circular_cosine,x_norm,y_norm
0,77.75,57000,2,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,960471.903756,1.942550e+06,1,0,0,1,0,0,25,1.666667,-0.866025,-0.500000,23.425601,6.173547
1,77.75,57000,2,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,960471.903756,1.942550e+06,1,0,0,1,0,0,25,1.916667,-0.500000,0.866025,23.425601,6.173547
2,77.75,35000,4,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,960471.903756,1.942550e+06,0,1,0,1,0,0,25,1.000000,0.000000,1.000000,23.425601,6.173547
3,77.75,32000,9,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,960471.903756,1.942550e+06,0,1,0,1,0,0,25,1.166667,0.866025,0.500000,23.425601,6.173547
4,77.75,37000,1,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,960471.903756,1.942550e+06,0,1,0,1,0,0,25,1.666667,-0.866025,-0.500000,23.425601,6.173547


In [38]:
main_data_df.tail()

,전용면적(㎡),가격(만원),층,공시지가(만원),토지면적,지목명_대,지목명_도로,지목명_임야,지목명_공원,지목명_학교용지,지목명_종교용지,지목명_주차장,지목명_철도용지,지목명_주유소용지,지목명_잡종지,지목명_수도용지,지목명_전,지목명_공장용지,지목명_창고용지,지목명_답,용도지역명1_제1종일반주거지역,용도지역명1_자연녹지지역,용도지역명1_개발제한구역,용도지역명1_제2종일반주거지역,용도지역명1_제3종일반주거지역,용도지역명1_일반상업지역,용도지역명1_준주거지역,용도지역명1_제1종전용주거지역,용도지역명1_중심상업지역,용도지역명1_근린상업지역,용도지역명1_준공업지역,용도지역명1_유통상업지역,용도지역명1_제2종전용주거지역,지형높이_평지,지형높이_완경사,지형높이_지정되지않음,지형높이_급경사,지형높이_고지,지형높이_저지,지형형상_사다리형,지형형상_부정형,지형형상_지정되지않음,지형형상_삼각형,지형형상_정방형,지형형상_가로장방,지형형상_역삼각형,지형형상_자루형,지형형상_세로장방,도로접면_소로한면,도로접면_지정되지않음,도로접면_맹지,도로접면_중로각지,도로접면_중로한면,도로접면_소로각지,도로접면_세로한면(가),도로접면_세로한면(불),도로접면_세로각지(불),도로접면_세로각지(가),도로접면_광대로한면,도로접면_광대세각,도로접면_광대소각,x좌표,y좌표,전월세매매구분_매매,전월세매매구분_전세,전월세매매구분_월세,건물종류_아파트,건물종류_오피스텔,건물종류_연립다세대,건물나이,time_linear,month_circular_sine,month_circular_cosine,x_norm,y_norm
3225081,19.32,500,4,353.5,217.9,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,962778.579431,1.957112e+06,0,0,1,0,1,0,17,8.583333,-0.500000,-8.660254e-01,25.732277,20.735379
3225082,28.98,6000,5,353.5,217.9,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,962778.579431,1.957112e+06,0,0,1,0,1,0,17,8.666667,-0.866025,-5.000000e-01,25.732277,20.735379
3225083,21.42,1000,2,353.5,217.9,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,962778.579431,1.957112e+06,0,0,1,0,1,0,17,8.666667,-0.866025,-5.000000e-01,25.732277,20.735379
3225084,31.50,3000,5,353.5,217.9,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,962778.579431,1.957112e+06,0,0,1,0,1,0,17,8.750000,-1.000000,-1.836970e-16,25.732277,20.735379
3225085,22.76,2000,3,370.1,372.9,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,964710.267920,1.955851e+06,0,0,1,0,1,0,8,8.000000,0.000000,1.000000e+00,27.663965,19.473954


In [41]:
main_data_df.drop(columns=['x좌표', 'y좌표'], inplace=True)

In [43]:
main_data_df.head()

,전용면적(㎡),가격(만원),층,공시지가(만원),토지면적,지목명_대,지목명_도로,지목명_임야,지목명_공원,지목명_학교용지,지목명_종교용지,지목명_주차장,지목명_철도용지,지목명_주유소용지,지목명_잡종지,지목명_수도용지,지목명_전,지목명_공장용지,지목명_창고용지,지목명_답,용도지역명1_제1종일반주거지역,용도지역명1_자연녹지지역,용도지역명1_개발제한구역,용도지역명1_제2종일반주거지역,용도지역명1_제3종일반주거지역,용도지역명1_일반상업지역,용도지역명1_준주거지역,용도지역명1_제1종전용주거지역,용도지역명1_중심상업지역,용도지역명1_근린상업지역,용도지역명1_준공업지역,용도지역명1_유통상업지역,용도지역명1_제2종전용주거지역,지형높이_평지,지형높이_완경사,지형높이_지정되지않음,지형높이_급경사,지형높이_고지,지형높이_저지,지형형상_사다리형,지형형상_부정형,지형형상_지정되지않음,지형형상_삼각형,지형형상_정방형,지형형상_가로장방,지형형상_역삼각형,지형형상_자루형,지형형상_세로장방,도로접면_소로한면,도로접면_지정되지않음,도로접면_맹지,도로접면_중로각지,도로접면_중로한면,도로접면_소로각지,도로접면_세로한면(가),도로접면_세로한면(불),도로접면_세로각지(불),도로접면_세로각지(가),도로접면_광대로한면,도로접면_광대세각,도로접면_광대소각,전월세매매구분_매매,전월세매매구분_전세,전월세매매구분_월세,건물종류_아파트,건물종류_오피스텔,건물종류_연립다세대,건물나이,time_linear,month_circular_sine,month_circular_cosine,x_norm,y_norm
0,77.75,57000,2,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,25,1.666667,-0.866025,-0.500000,23.425601,6.173547
1,77.75,57000,2,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,25,1.916667,-0.500000,0.866025,23.425601,6.173547
2,77.75,35000,4,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,25,1.000000,0.000000,1.000000,23.425601,6.173547
3,77.75,32000,9,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,25,1.166667,0.866025,0.500000,23.425601,6.173547
4,77.75,37000,1,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,25,1.666667,-0.866025,-0.500000,23.425601,6.173547


In [55]:
def coor_fix(df):
    gc.collect()
    df = df.copy()
    
    x_min = df['x_norm'].min()
    x_max = df['x_norm'].max()
    x_middle = (x_max - x_min) /2
    
    y_min = df['y_norm'].min()
    y_max = df['y_norm'].max()
    y_middle = (y_max - y_min) /2
    
    df['x_norm'] = (df['x_norm'] - x_min - x_middle)
    df['y_norm'] = (df['y_norm'] - y_min - y_middle)
    
    return df

In [51]:
main_data_df = coor_fix(main_data_df)

In [52]:
main_data_df.head()

,전용면적(㎡),가격(만원),층,공시지가(만원),토지면적,지목명_대,지목명_도로,지목명_임야,지목명_공원,지목명_학교용지,지목명_종교용지,지목명_주차장,지목명_철도용지,지목명_주유소용지,지목명_잡종지,지목명_수도용지,지목명_전,지목명_공장용지,지목명_창고용지,지목명_답,용도지역명1_제1종일반주거지역,용도지역명1_자연녹지지역,용도지역명1_개발제한구역,용도지역명1_제2종일반주거지역,용도지역명1_제3종일반주거지역,용도지역명1_일반상업지역,용도지역명1_준주거지역,용도지역명1_제1종전용주거지역,용도지역명1_중심상업지역,용도지역명1_근린상업지역,용도지역명1_준공업지역,용도지역명1_유통상업지역,용도지역명1_제2종전용주거지역,지형높이_평지,지형높이_완경사,지형높이_지정되지않음,지형높이_급경사,지형높이_고지,지형높이_저지,지형형상_사다리형,지형형상_부정형,지형형상_지정되지않음,지형형상_삼각형,지형형상_정방형,지형형상_가로장방,지형형상_역삼각형,지형형상_자루형,지형형상_세로장방,도로접면_소로한면,도로접면_지정되지않음,도로접면_맹지,도로접면_중로각지,도로접면_중로한면,도로접면_소로각지,도로접면_세로한면(가),도로접면_세로한면(불),도로접면_세로각지(불),도로접면_세로각지(가),도로접면_광대로한면,도로접면_광대세각,도로접면_광대소각,전월세매매구분_매매,전월세매매구분_전세,전월세매매구분_월세,건물종류_아파트,건물종류_오피스텔,건물종류_연립다세대,건물나이,time_linear,month_circular_sine,month_circular_cosine,x_norm,y_norm
0,77.75,57000,2,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,25,1.666667,-0.866025,-0.500000,55.40076,-90.586155
1,77.75,57000,2,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,25,1.916667,-0.500000,0.866025,55.40076,-90.586155
2,77.75,35000,4,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,25,1.000000,0.000000,1.000000,55.40076,-90.586155
3,77.75,32000,9,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,25,1.166667,0.866025,0.500000,55.40076,-90.586155
4,77.75,37000,1,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,25,1.666667,-0.866025,-0.500000,55.40076,-90.586155


In [53]:
main_data_df['x_norm'].describe()

count    3.225086e+06
mean     2.609364e+00
std      8.120275e+01
min     -1.688553e+02
25%     -6.832351e+01
50%      2.081076e+01
75%      6.775403e+01
max      1.688553e+02
Name: x_norm, dtype: float64

In [54]:
main_data_df['y_norm'].describe()

count    3.225086e+06
mean    -1.752461e+01
std      5.875554e+01
min     -1.423216e+02
25%     -6.623635e+01
50%     -2.564290e+01
75%      2.000146e+01
max      1.423216e+02
Name: y_norm, dtype: float64

In [56]:
main_data_df.to_csv('training_data.csv', index=False)